In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import KNNImputer

In [2]:
# reads data from file
diseases = pd.read_csv('proj-data.csv')

# defines possible values for each field
hyperthyroid_conditions = ['A', 'B', 'C', 'D']
hypothyroid_conditions = ['E', 'F', 'G', 'H']
binding_protein = ['I', 'J']
general_health = ['K']
replacement_therapy = ['L', 'M', 'N']
discordant_results = ['R']

# if it has any of the values above, relates to the corresponding field
for d in range(len(diseases)):
    if diseases.at[d, "diagnoses"] in hyperthyroid_conditions :
        diseases.at[d, "diagnoses"] = 1
    elif diseases.at[d, "diagnoses"] in hypothyroid_conditions :
        diseases.at[d, "diagnoses"] = 2
    elif diseases.at[d, "diagnoses"] in binding_protein :
        diseases.at[d, "diagnoses"] = 3
    elif diseases.at[d, "diagnoses"] in general_health :
        diseases.at[d, "diagnoses"] = 4
    elif diseases.at[d, "diagnoses"] in replacement_therapy :
        diseases.at[d, "diagnoses"] = 5
    elif diseases.at[d, "diagnoses"] in discordant_results :
        diseases.at[d, "diagnoses"] = 6
    elif diseases.at[d, "diagnoses"] == "-" :
        diseases.at[d, "diagnoses"] = 7 # no condition
    else:
        diseases.at[d, "diagnoses"] = 8 # others


# changes certain values to 0 and 1s
diseases.drop('referral source:',axis=1, inplace=True) # I think this is not relevant TODO check to make sure (assign an int to each source)

diseases.replace('?', np.nan, inplace=True)
diseases.replace('f', 0, inplace=True)
diseases.replace('t', 1, inplace=True)
diseases.replace('F', 0, inplace=True)
diseases.replace('M', 1, inplace=True)

diseases

,age:,sex:,on thyroxine:,query on thyroxine:,on antithyroid medication:,sick:,pregnant:,thyroid surgery:,I131 treatment:,query hypothyroid:,...,TT4 measured:,TT4:,T4U measured:,T4U:,FTI measured:,FTI:,TBG measured:,TBG:,diagnoses,[record identification]
0,29,0.0,0,0,0,0,0,0,0,1,...,0,NaN,0,NaN,0,NaN,0,NaN,7,[861106018]
1,29,0.0,0,0,0,0,0,0,0,0,...,1,128,0,NaN,0,NaN,0,NaN,7,[860916073]
2,36,0.0,0,0,0,0,0,0,0,0,...,0,NaN,0,NaN,0,NaN,1,26,7,[850726049]
3,60,0.0,0,0,0,0,0,0,0,0,...,0,NaN,0,NaN,0,NaN,1,26,7,[861010020]
4,77,0.0,0,0,0,0,0,0,0,0,...,0,NaN,0,NaN,0,NaN,1,21,7,[860324074]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7333,56,1.0,0,0,0,0,0,0,0,0,...,1,64,1,0.83,1,77,0,NaN,7,[850725042]
7334,22,1.0,0,0,0,0,0,0,0,0,...,1,91,1,0.92,1,99,0,NaN,7,[840816047]
7335,69,1.0,0,0,0,0,0,0,0,0,...,1,113,1,1.27,1,89,0,NaN,3,[840910050]
7336,47,0.0,0,0,0,0,0,0,0,0,...,1,75,1,0.85,1,88,0,NaN,7,[860317010]


In [3]:

#col_names = np.array(diseases.columns)[0:28]

#print("os nomes das colunas são: ", col_names)

# Define columns for X and Y TODO change to use all data instead of 500 collumns
X = diseases.values[:,:27]
y = diseases.values[:,28]


# SPLIT INTO TRAIN AND TEST DATA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

# Transforms the y train and test into 1 dimensional arrays
y_train = y_train.squeeze().ravel().astype('int')
y_test = y_test.squeeze().ravel().astype('int')


# SCALING - It's not used, mostly breaks the data
scaler = StandardScaler()
scaler.fit(X_train)
Xs_train=scaler.fit_transform(X_train)
Xs_test=scaler.fit_transform(X_test)
# Não é usado

# IMPUTER - Normalizes data, so that it's well formatted
# eliminates the '?' and such
imputer = KNNImputer(weights="uniform")
imputer.fit(X_train)

# new values for train and test data after imputer
Xi_train = imputer.transform(X_train)
Xi_test = imputer.transform(X_test)


# check if there's more data
print(len(Xi_train), ' vs ', len(X_train))
print(len(Xi_test), ' vs ', len(X_test))


4916  vs  4916
2422  vs  2422


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, make_scorer

# Function that prints the class statistics
# TRUTH_nfold are the actual values, PREDS_nfold are the predicted values
def  printClaStatistics(TRUTH_nfold, PREDS_nfold):   
    print("These are the training set statistics:")
    print("The Accuracy is: %7.4f" % accuracy_score(TRUTH_nfold, PREDS_nfold))
    print("The Precision is: %7.4f" % precision_score(TRUTH_nfold, PREDS_nfold, average='weighted'))
    print("The Recall is: %7.4f" % recall_score(TRUTH_nfold, PREDS_nfold, average='weighted'))
    print("The F1 score is: %7.4f" % f1_score(TRUTH_nfold, PREDS_nfold, average='weighted'))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(TRUTH_nfold, PREDS_nfold))
    print(pd.DataFrame(confusion_matrix(TRUTH_nfold, PREDS_nfold)))

## Categorical Naive Bayes - wwrong, usar Gaussian Bayes 

In [13]:
from sklearn.naive_bayes import CategoricalNB

# creates the model 
#mdl = CategoricalNB(alpha=0.001)
mdl=GaussianNB() 

# train the model
mdl.fit(Xi_train, y_train)

# makes predictions
print("Shape of X_train:", Xi_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", Xi_test.shape)

predsBayes=mdl.predict(Xi_test) #ocorre erro aqui A CAUSA É QUE NAO SE PODE USAR O CATEGORICAL BAYES

# prints case '-->' prediction
for i, p in enumerate(preds):
    print("Case: ", i, '-->' , p) #, "-->", p)


# prints statistics
printClaStatistics(y_test, predsBayes)

Shape of X_train: (4916, 27)
Shape of y_train: (4916,)
Shape of x_test: (2422, 27)
Case:  0 --> 6
Case:  1 --> 6
Case:  2 --> 1
Case:  3 --> 6
Case:  4 --> 6
Case:  5 --> 6
Case:  6 --> 6
Case:  7 --> 5
Case:  8 --> 6
Case:  9 --> 4
Case:  10 --> 5
Case:  11 --> 6
Case:  12 --> 2
Case:  13 --> 4
Case:  14 --> 3
Case:  15 --> 4
Case:  16 --> 6
Case:  17 --> 4
Case:  18 --> 6
Case:  19 --> 6
Case:  20 --> 8
Case:  21 --> 6
Case:  22 --> 4
Case:  23 --> 2
Case:  24 --> 6
Case:  25 --> 5
Case:  26 --> 6
Case:  27 --> 4
Case:  28 --> 6
Case:  29 --> 5
Case:  30 --> 1
Case:  31 --> 2
Case:  32 --> 4
Case:  33 --> 1
Case:  34 --> 1
Case:  35 --> 3
Case:  36 --> 5
Case:  37 --> 6
Case:  38 --> 6
Case:  39 --> 2
Case:  40 --> 6
Case:  41 --> 5
Case:  42 --> 7
Case:  43 --> 4
Case:  44 --> 6
Case:  45 --> 6
Case:  46 --> 6
Case:  47 --> 6
Case:  48 --> 1
Case:  49 --> 6
Case:  50 --> 6
Case:  51 --> 7
Case:  52 --> 6
Case:  53 --> 6
Case:  54 --> 6
Case:  55 --> 5
Case:  56 --> 6
Case:  57 --> 5

Does not accept NaNs, so we need to impute new values, so the data used isn't true to the real data and might compromise the results

## Gaussian Naive Bayes

In [15]:
#from sklearn.naive_bayes import GaussianNB só para não termos a mesma coisa duas vezes

#mdl = GaussianNB(alpha=0.001)

#print(type(xs_train[3][28]))

#mdl.fit(xs_train,y_train)

#preds=mdl.predict(xs_test)
#for i, p in enumerate(preds):
    #print("Case: ", xs_test[i], "-->", p) 

## Decision Tree Classifier

In [21]:
from sklearn.tree import DecisionTreeClassifier

# creates the model
clf = DecisionTreeClassifier(random_state=69)

#trains the model
clf.fit(Xi_train, y_train)

# makes predictions
predsDTC = clf.predict(Xi_test)

# prints statistics
printClaStatistics(y_test, predsDTC)

These are the training set statistics:
The Accuracy is:  0.9141
The Precision is:  0.9178
The Recall is:  0.9141
The F1 score is:  0.9151
The Matthews correlation coefficient is:  0.8075
    0    1   2   3   4   5     6   7
0  28    0   6   0   1   7     9   1
1   0  150   0   0   0   0     1   2
2   1    0  75   2   2   1    17   2
3   0    0   0  98   0   0    20   1
4   0    0   1   0  77   0     1   2
5   0    0   0   1   0  50     5   4
6   7    4  25  25   7  10  1701  22
7   0    3   0   1   0   1    16  35


## K-NNeighbors Classifier

In [25]:
from sklearn.neighbors import KNeighborsClassifier

# creates the model
knn_classifier = KNeighborsClassifier()

#trains the model
knn_classifier.fit(Xi_train, y_train)

# makes predictions
predsKNN = knn_classifier.predict(Xi_test)

# prints statistics
printClaStatistics(y_test, predsKNN)

In [ ]:
# Modelo de regressão linear "Linear Regression": LinearRegression(),  
# Modelo de regressão de árvore de decisão    "Decision Tree Regressor": DecisionTreeRegressor(),  
# Modelo de regressão k-NN    "k-NN Regressor": KNeighborsRegressor(),  
# Modelo de regressão de vetor de suporte (SVR)    "SVR": SVR(),  
# Modelo de regressão de floresta aleatória    "Random Forest Regressor": RandomForestRegressor()  